# Datamodel and Database module

The TIdatabase module encapsulates all loading, storing and joining of the student, college and applications dataframes.

The module will be imported at the beginning of every iPython Notebook.

In [1]:
import TIdatabase as ti
%matplotlib inline 
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from matplotlib import rcParams

## The Datamodel
This [Google Docs](https://docs.google.com/spreadsheets/d/1dm73Vmov8bhNoVRUtyg6TU-IgE7DPDVlukMkvnaCqAg/edit#gid=0&vpid=A1) contains a list of what we believe are the important factors in the college decision. This is list of course does not include things as recommendation letters as it is impossible to get data for this or to quantifiy it. The doc also includes the column names that each feature has our dataframes. We distinguish 3 dataframes:
- A students dataframe contains all academic and personal data of a particular student (scores, gender, etc)
- A college dataframe contains all information of a university (acceptance rate, public/private, etc)
- An applications dataframe contains application-specific data for a particular student in a particular university, for example and most importantly, the result of the decision procedure

## Generating Mock Data

The module has the functionality to fill the dataframes with mock data, which is useful to start writing  classification code before we finish scraping the actual data.

In [2]:
students = ti.Student()
# populate students with random values
students.fillRandom(10)
students.df 

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp,schooltype
0,B7BYUBJTLU,0.663893,0.020092,0.096024,0.731044,0.558752,0.370973,0.368899,TDWJCGUNSOLT,2012,...,0,1,0,0,0,0,1,0,1,0
1,JRTWQBISJJ,0.518537,0.192981,0.892634,0.116132,0.108287,0.723827,0.503420,TGVSHSVYQFRV,2010,...,0,0,1,0,0,0,1,1,0,0
2,3PVVZQEF76,0.082301,0.264000,0.125186,0.250936,0.125729,0.513196,0.139161,JXTLXUJDVDLP,2014,...,1,0,1,0,0,1,0,1,1,1
3,M84BLJVFPF,0.657374,0.045096,0.715776,0.257288,0.085336,0.695988,0.348862,QBKPXOZHNLEQ,2011,...,0,1,0,1,0,0,1,0,0,1
4,F79BWOEM7Q,0.247324,0.274292,0.370158,0.251320,0.833662,0.015930,0.371374,RTEDFMZCQGIO,2018,...,1,1,0,0,1,1,0,0,1,1
5,UF4HKHKR8W,0.030254,0.498338,0.764240,0.848219,0.682040,0.096465,0.677315,FWZGOFMNZSSR,2017,...,1,0,0,0,1,0,0,0,0,1
6,0EIHE2BUBI,0.199104,0.756616,0.968281,0.699453,0.266624,0.995723,0.286685,NFEHDEOWDCZO,2017,...,1,1,1,1,1,0,1,0,1,0
7,O0RDXZYD6E,0.840712,0.805452,0.991282,0.520968,0.482520,0.060888,0.899072,GRPMFAZUJCXY,2019,...,1,1,1,1,0,0,0,1,0,1
8,EUC203C0GS,0.440727,0.822046,0.806502,0.099229,0.475223,0.298240,0.109659,QMFADWBOATMI,2013,...,0,1,0,1,0,0,1,1,1,0
9,7NLIYJ1JFZ,0.901041,0.679504,0.419757,0.892776,0.298304,0.485504,0.513853,IJYWLFOPSPGN,2019,...,1,0,1,1,0,1,0,0,1,1


#### Simulating Missing Data

You can also simulate NaNs in the mockup. `fillRandom` takes a second optional parameter that is the percentage of NaNs to generate.

In [3]:
students.fillRandom(10, 0.25) # 25% of values will be NaN
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp,schooltype
0,AR2WOBI3V0,0.043604,NaN,0.998352,NaN,0.138451,NaN,NaN,PMCUQILHRANQ,2016,...,1,1,NaN,1,1,0,NaN,1,0,1
1,8B9M5C1KFA,0.392687,0.844994,0.543490,NaN,0.721894,0.771651,0.730015,NaN,2017,...,NaN,1,NaN,NaN,1,0,0,1,1,1
2,H9FG174Y2Z,0.774163,NaN,0.455203,NaN,0.944819,0.241725,NaN,NJQSBQDBEZQP,2013,...,1,0,NaN,1,0,1,1,0,1,1
3,NK5HTHCJ6N,NaN,0.828176,0.229793,0.185385,0.307424,NaN,NaN,SUNDIQQMDYCM,2012,...,1,NaN,0,0,1,NaN,NaN,1,1,1
4,7RF77SSXS3,0.404155,0.148635,0.802617,0.455309,NaN,0.526411,NaN,SAUTGMCOWGBN,2010,...,1,1,0,0,0,NaN,1,NaN,NaN,0
5,9JWTP1PI4U,NaN,0.801332,0.253159,0.074525,0.868009,0.701850,0.091131,ZVOKUZQKRLDB,NaN,...,0,1,1,NaN,1,0,1,NaN,NaN,0
6,N55RAAKVS7,0.062926,0.230727,NaN,0.473766,0.233258,NaN,0.667772,ZAKMFRCZTLGV,2015,...,NaN,1,NaN,1,NaN,0,1,0,NaN,1
7,16MWKFI2MU,0.988868,0.494548,0.889030,NaN,0.028554,0.522539,NaN,FDBTDNZENJNX,NaN,...,NaN,1,1,NaN,0,0,1,1,1,0
8,49UID52HLC,0.462005,0.766104,NaN,0.361330,0.196404,0.222940,0.743974,BGTFYYPKPKBD,NaN,...,0,0,1,NaN,1,NaN,NaN,NaN,0,0
9,QKA3C4AFQT,0.342439,0.115699,0.325553,0.873661,NaN,0.562489,NaN,LQZBEXPKGVHW,NaN,...,NaN,0,0,0,1,0,1,NaN,1,1


Since we have a fixed list of only 25 colleges, the college infromation is not scraped.  The list of colleges is populated from a CSV stored in the same directory as this notebook. It can be edited using your favorite CSV editor, such as Excel. When you create a new instance of College, the values will be read in from the CSV. 

In [4]:
# populate with list of known colleges
colleges = ti.College()
colleges.df

,collegeID,name,acceptrate,size,public,finAidPct,instatePct
0,Princeton,Princeton,0.074,5142,-1,0.00,0
1,Harvard,Harvard,0.060,19929,-1,0.75,0
2,Yale,Yale,0.063,12336,-1,0.00,0
3,Columbia,Columbia,0.070,24221,-1,0.00,0
4,Stanford,Stanford,0.051,16795,-1,0.00,0
5,UChicago,UChicago,0.088,12558,-1,0.00,0
6,MIT,MIT,0.079,11319,-1,0.00,0
7,Duke,Duke,0.114,15856,-1,0.00,0
8,UPenn,UPenn,0.104,21296,-1,0.00,0
9,CalTech,CalTech,0.088,2209,-1,0.00,0


The table of application forms combines a student with a university and carries the information of specific applications. `acceptStatus` is our predictor. `acceptProb` is our $\hat{Y}$ probability.

In [5]:
applForm = ti.ApplForm()
applForm.fillRandom(30)
applForm.df

,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,16MWKFI2MU,NotreDame,0,0,0,1,1,0.517955
1,49UID52HLC,NotreDame,1,1,0,1,1,0.759026
2,49UID52HLC,Rice,1,1,0,0,1,0.456000
3,N55RAAKVS7,Georgetown,1,0,1,0,0,0.072097
4,AR2WOBI3V0,Columbia,1,1,0,0,1,0.119545
5,8B9M5C1KFA,Vanderbilt,1,1,0,1,1,0.060323
6,H9FG174Y2Z,UCLA,1,0,0,0,0,0.616532
7,9JWTP1PI4U,Vanderbilt,0,0,0,0,1,0.881892
8,8B9M5C1KFA,Yale,0,0,1,0,1,0.062462
9,H9FG174Y2Z,Georgetown,0,0,1,0,0,0.116222


To combine the student and application forms tables, we use Pandas merge capability, which will match rows by identical column names, which is studentID in this case:

In [6]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,artist,workexp,schooltype,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,AR2WOBI3V0,0.043604,NaN,0.998352,NaN,0.138451,NaN,NaN,PMCUQILHRANQ,2016,...,1,0,1,Columbia,1,1,0,0,1,0.119545
1,AR2WOBI3V0,0.043604,NaN,0.998352,NaN,0.138451,NaN,NaN,PMCUQILHRANQ,2016,...,1,0,1,Cornell,1,0,1,0,1,0.322915
2,AR2WOBI3V0,0.043604,NaN,0.998352,NaN,0.138451,NaN,NaN,PMCUQILHRANQ,2016,...,1,0,1,UChicago,0,1,0,1,1,0.567666
3,8B9M5C1KFA,0.392687,0.844994,0.543490,NaN,0.721894,0.771651,0.730015,NaN,2017,...,1,1,1,Vanderbilt,1,1,0,1,1,0.060323
4,8B9M5C1KFA,0.392687,0.844994,0.543490,NaN,0.721894,0.771651,0.730015,NaN,2017,...,1,1,1,Yale,0,0,1,0,1,0.062462
5,8B9M5C1KFA,0.392687,0.844994,0.543490,NaN,0.721894,0.771651,0.730015,NaN,2017,...,1,1,1,Emory,0,0,0,1,0,0.033881
6,8B9M5C1KFA,0.392687,0.844994,0.543490,NaN,0.721894,0.771651,0.730015,NaN,2017,...,1,1,1,JohnsHopkins,1,1,0,0,1,0.521624
7,8B9M5C1KFA,0.392687,0.844994,0.543490,NaN,0.721894,0.771651,0.730015,NaN,2017,...,1,1,1,MIT,1,0,1,0,1,0.530897
8,H9FG174Y2Z,0.774163,NaN,0.455203,NaN,0.944819,0.241725,NaN,NJQSBQDBEZQP,2013,...,0,1,1,UCLA,1,0,0,0,0,0.616532
9,H9FG174Y2Z,0.774163,NaN,0.455203,NaN,0.944819,0.241725,NaN,NJQSBQDBEZQP,2013,...,0,1,1,Georgetown,0,0,1,0,0,0.116222


Now the `applications` Pandas DataFrame is ready to use for either regression (by overwriting the acceptProb column) or building the public facing web site.

## Saving Scraped Data

### Part 1 - The Student Data

First, let's start fresh and delete the previously created objects. This is only necessary because this sample script is running within Jupyter where all variables are global to the page. In a separate Python file run from the command line, this step can be skipped. Creating a new instance would not work as behind the scenes, there is sharing of
DataFrames between objects.

In [7]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm


Let's create a new students instance. It will be an empty Pandas dataframe with the correct columns

In [8]:
students = ti.Student()
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp,schooltype


Populate a dictionary with the values that the scraper has for a given row. Make sure the keys match up with the column names as only the matching columns will be saved. There is no need to add the studentID key. A unique value will be generated automatically and returned from the insert. This will be in the same order as the provided rows. Saving the generated student IDs will be helpful later when populating the applForm foreign key.

In [9]:

# Example: international male who scored in 98th percentile in ACT/SAT, went to a public school and is applying for
# Class of 2020

newrow = {'admissionstest': 0.98,
         'GPA': 4.5,
         'female' : -1,
         'international': 1,
         'schooltype': -1,
         'intendedgradyear':2020}

newsinglestudentID = students.insert(newrow)
print "New studentID:",newsinglestudentID
students.df

New studentID: ['WB1OBL2P44']


,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp,schooltype
0,WB1OBL2P44,NaN,0.98,NaN,NaN,NaN,4.5,NaN,NaN,2020,...,NaN,-1,NaN,NaN,1,NaN,NaN,NaN,NaN,-1


It is more efficient if multiple rows are added in one step. In this case, create a list of dictionaries and just
use the same method. Here, two new rows are added to the DataFrame in one step.

In [10]:
rows = []
a = {'schooltype': -1, 'admissionstest': 0.98, 'GPA': 4.5, 'female': 1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
a = {'schooltype': 1, 'admissionstest': 0.65, 'GPA': 2.2, 'female': -1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
newmanystudentIDs = students.insert(rows)
print "New studentIDs:",newmanystudentIDs
students.df

New studentIDs: ['K0B4WHJPXR', '6C9VBG477L']


,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp,schooltype
0,WB1OBL2P44,NaN,0.98,NaN,NaN,NaN,4.5,NaN,NaN,2020,...,NaN,-1,NaN,NaN,1,NaN,NaN,NaN,NaN,-1
0,K0B4WHJPXR,NaN,0.98,NaN,NaN,NaN,4.5,NaN,NaN,2019,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,-1
1,6C9VBG477L,NaN,0.65,NaN,NaN,NaN,2.2,NaN,NaN,2019,...,NaN,-1,NaN,NaN,0,NaN,NaN,NaN,NaN,1


Now we are ready to save. The data is saved in CSV format for ease of interpretability.

In [11]:
students.save("mydata.csv")

Let's delete all the data and check that we can read it back successfully.

In [12]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm



In [13]:
students = ti.Student()
students.read("mydata.csv")
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,canAfford,female,MinorityGender,MinorityRace,international,firstinfamily,sports,artist,workexp,schooltype
0,WB1OBL2P44,NaN,0.98,NaN,NaN,NaN,4.5,NaN,NaN,2020,...,NaN,-1,NaN,NaN,1,NaN,NaN,NaN,NaN,-1
0,K0B4WHJPXR,NaN,0.98,NaN,NaN,NaN,4.5,NaN,NaN,2019,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,-1
1,6C9VBG477L,NaN,0.65,NaN,NaN,NaN,2.2,NaN,NaN,2019,...,NaN,-1,NaN,NaN,0,NaN,NaN,NaN,NaN,1


Et, voilà, the data is back.

### Part 2 - The Application Data

This is pretty much the same, **except** for two important differences:

* The studentID and collegeID must both be populated and exist in the respective DataFrames
* The columns are a little different. Normally acceptProb would not be populated from the scraper but could be used to store prediction runs. 

In [14]:
# we already wiped out applForm above
applForm = ti.ApplForm()
applForm.df

,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb


In [15]:
# either pick one from the students.df DataFrame, like this:
#--- studentID = students.df.iloc[1].studentID
# or use the studentID from the insert in the students DataFrame
studentID = newsinglestudentID[0]
collegeID = colleges.df.iloc[1].collegeID

In [16]:
print studentID, collegeID

WB1OBL2P44 Harvard


In [17]:
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}

applForm.insert(newrow)
applForm.df

,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,WB1OBL2P44,Harvard,0,0,NaN,NaN,0,NaN


Now inserting multiple rows

In [18]:
rows = []
# either pick one randomly:
#--- studentID = students.df.iloc[2].studentID
# or use the list from the insert into the students DataFrame
# iterate over the list of newstudentIDs if necessary
studentID = newmanystudentIDs[0]
collegeID = colleges.df.iloc[2].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}
rows.append(newrow)
# note: same student, new school to apply to
studentID = students.df.iloc[2].studentID
collegeID = colleges.df.iloc[3].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 1,
         'visited': 1,
         'acceptStatus': 1}
rows.append(newrow)
applForm.insert(rows)
applForm.df

,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,WB1OBL2P44,Harvard,0,0,NaN,NaN,0,NaN
0,K0B4WHJPXR,Yale,0,0,NaN,NaN,0,NaN
1,6C9VBG477L,Columbia,1,1,NaN,NaN,1,NaN


Let's save it

In [19]:
applForm.save("applform1.csv")

Then delete the local variable

In [20]:
if ('applForm' in locals()): del applForm

Then read it back

In [21]:
applForm = ti.ApplForm()
applForm.read("applform1.csv")
applForm.df


,studentID,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,WB1OBL2P44,Harvard,0,0,NaN,NaN,0,NaN
0,K0B4WHJPXR,Yale,0,0,NaN,NaN,0,NaN
1,6C9VBG477L,Columbia,1,1,NaN,NaN,1,NaN


And now let's check that the merge still works

In [22]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,GPA_w,program,intendedgradyear,...,artist,workexp,schooltype,collegeID,earlyAppl,visited,alumni,outofstate,acceptStatus,acceptProb
0,WB1OBL2P44,NaN,0.98,NaN,NaN,NaN,4.5,NaN,NaN,2020,...,NaN,NaN,-1,Harvard,0,0,NaN,NaN,0,NaN
1,K0B4WHJPXR,NaN,0.98,NaN,NaN,NaN,4.5,NaN,NaN,2019,...,NaN,NaN,-1,Yale,0,0,NaN,NaN,0,NaN
2,6C9VBG477L,NaN,0.65,NaN,NaN,NaN,2.2,NaN,NaN,2019,...,NaN,NaN,1,Columbia,1,1,NaN,NaN,1,NaN


Ok, we are done for today.